### Build concat TS and connectome with chunks

In [19]:
import os
import pickle
import numpy as np
from nilearn import connectome
from sklearn.covariance import LedoitWolf

# === Configuration ===
dimension_of_atlas = 128
chunked_dir = "128"
min_timepoints = 10
connectome_method = "tangent"
discard_diagonal = False

# === Load chunked data ===
all_chunks = []
all_chunk_ids = []

for filename in os.listdir(chunked_dir):
    if filename.endswith(".pkl"):
        subject_id = filename.replace(".pkl", "")
        with open(os.path.join(chunked_dir, filename), "rb") as f:
            chunks = pickle.load(f)
        for idx, chunk in enumerate(chunks):
            if chunk.shape[0] < min_timepoints:
                continue
            all_chunks.append(chunk)
            all_chunk_ids.append(f"{subject_id}_chunk{idx+1}")

print(f"✅ Loaded {len(all_chunks)} usable chunks from {len(all_chunk_ids)} total chunk IDs.")

# === Save all time series (with IDs) ===
ts_output_path = f"Results/128/ts_{dimension_of_atlas}_with_ids.pkl"
with open(ts_output_path, "wb") as f:
    pickle.dump((all_chunks, all_chunk_ids), f)
print(f"📦 Saved concatenated time series to {ts_output_path}")

# === Compute connectomes ===
connectome_measure = connectome.ConnectivityMeasure(
    cov_estimator=LedoitWolf(assume_centered=True),
    kind=connectome_method,
    discard_diagonal=discard_diagonal,
    vectorize=True
)

X = connectome_measure.fit_transform(all_chunks)
print(f"✅ Computed connectomes with shape: {X.shape}")

# === Save connectomes and IDs ===
connectome_path = f"Results/128/connectomes_{dimension_of_atlas}.npy"
ids_path = f"Results/128/connectome_ids_{dimension_of_atlas}.pkl"

np.save(connectome_path, X)
with open(ids_path, "wb") as f:
    pickle.dump(all_chunk_ids, f)

print(f"📦 Saved vectorized connectomes to {connectome_path}")
print(f"📦 Saved corresponding IDs to {ids_path}")


✅ Loaded 3410 usable chunks from 3410 total chunk IDs.
📦 Saved concatenated time series to Results/128/ts_128_with_ids.pkl
✅ Computed connectomes with shape: (3410, 8256)
📦 Saved vectorized connectomes to Results/128/connectomes_128.npy
📦 Saved corresponding IDs to Results/128/connectome_ids_128.pkl
